The purpose of this notebook is to loop through the rows of a dataset and and calculate the amount of time that the ferry was delayed for each row. Then a new column is created with this data.

In [1]:
import pandas as pd

df = pd.read_csv("../Cleaned-Data/Bainbridge_Departures_2013_Indexed.csv", parse_dates = ["timestamps"], index_col = "timestamps")

In [2]:
df.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime
timestamps,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN
2013-01-01 07:05:00,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN


In [4]:
def minutesToSeconds(minutes):
    if(":" in minutes):
        m, s = minutes.split(':')
        return (int(m)*60)+int(float(s))
    return (int(minutes)*60)

def pastHourDepartureDifference(actual_departure_minutes, scheduled_departure_minutes):
    # Calculate how far from end of the hour schedule_departure_minutes is
    actual_departure_past_hour = (60*60) - scheduled_departure_minutes
    
    # Add this on to the actual departure minutes
    return actual_departure_past_hour + actual_departure_minutes

In [17]:
def calculate_seconds_late(df):
    seconds_late = []

    for index, row in df.iterrows():

        if(not str(row.iloc[3]) == "nan"):
            actual_departure_seconds = minutesToSeconds(row.iloc[3])
        else:
            seconds_late.append(float('nan'))
            continue

        scheduled_departure_seconds = minutesToSeconds((row.iloc[2].split(":")[0]))

        # ex: actual_departure (05:56.0), scheduled_departure (0:55:00)
        if(actual_departure_seconds < scheduled_departure_seconds):
            difference = abs(actual_departure_seconds - scheduled_departure_seconds)

            # Making the assumption here that more than 5 minutes early is not possible.
            if(difference > (5*60)):
                seconds_late.append(pastHourDepartureDifference(actual_departure_seconds, scheduled_departure_seconds))
            #If less than 5 minutes then lets take it
            else:
                seconds_late.append((actual_departure_seconds - scheduled_departure_seconds))      
        elif(actual_departure_seconds > scheduled_departure_seconds):
            seconds_late.append(actual_departure_seconds - scheduled_departure_seconds)
        elif(actual_departure_seconds == scheduled_departure_seconds):
            seconds_late.append(0)
            
    return seconds_late

In [18]:
bainbrigde_2013_seconds_late = calculate_seconds_late(df)

In [19]:
len(bainbrigde_2013_seconds_late)

8234

In [20]:
df.shape

(8234, 7)

In [21]:
df["seconds_late"] = seconds_late

In [22]:
df.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0,656.0
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0,-59.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN,NaN
2013-01-01 07:05:00,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN,76.0


In [23]:
df.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-12-31 19:10:00,Bainbridge,Colman,10:00.0,10:03.0,44:26.0,34.0,3.0
2013-12-31 20:10:00,Bainbridge,Colman,10:00.0,10:53.0,41:57.0,31.0,53.0
2013-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:00.0,26:36.0,31.0,0.0
2013-12-31 21:45:00,Bainbridge,Colman,45:00.0,45:17.0,17:09.0,32.0,17.0
2013-12-31 22:35:00,Bainbridge,Colman,35:00.0,38:45.0,13:36.0,35.0,225.0


In [24]:
df.to_csv("Bainbridge_Departures_2013_with_Seconds_Late.csv")